In [3]:
import numpy as np
import nmrglue
import pandas as pd
def analyze_dataset(all_data):
    # Create a DataFrame with real, imaginary, and time values
    time = np.linspace(0, AQ, len(all_data))
    df = pd.DataFrame({
        "Time": time,
        "Real": all_data.real,
        "Imaginary": all_data.imag
    })
    all_data=nmrglue.proc_base.ps(all_data, p0+90, p1) #Why is there an extra 90 here? Probably from ph31 convention?
    # Apply exponential apodization
    #apodized_data = nmrglue.proc_base.em(all_data, .0004)
    #*********this function can only take real part and not full complex part. Do we want to edit this way?

    ls_and_apodized_data = nmrglue.proc_base.ls(all_data, 63)
    # Plot the results
    
    spectrum = nmrglue.proc_base.fft(ls_and_apodized_data) 
    n = len(all_data.real)
    freq = np.linspace(-SWH / 2+center, SWH / 2+center, n) #I am not sure how to calcualte the bounds of the FT here. But this mehtod lines up with Bruker.
    return (freq, np.real(spectrum), np.imag(spectrum))
    

In [4]:
# Important Acqus Parameters
AQ=1.6# Acquistion time (s)
SWH=2000 #Spectral Width [Hz]
SF=202.4765750 #Spectrometer frequency [Mhz]
SFO1=202.4751988 #Transmitter frequency [Mhz]
p0=-40
p1=0
center=-(SF-SFO1)*10**6
plot_width=30

In [5]:
file_paths = [
    fr"C:\Users\donny\Desktop\desktop_files\Brown\All_data\Final_Organized_MIT_DATA\2024_12_23-24\NOT_TOMOGRAPHY\ns4_analyze_phase_addition_p04Wdecoupler\{i}"
    for i in range(119, 174+1)  # Adjust the range as needed
]

# Initialize a list to store results
all_results = []

for path in file_paths:
    # Read the data
    all_data = nmrglue.bruker.read(path)[1]

    # Unpack arrays
    array0, array1, array2, _ = all_data

    # Process each array
    rho_0 = analyze_dataset(array0)
    rho_1 = analyze_dataset(array1)
    rho_2 = analyze_dataset(array2)

    # Combine processed arrays: real and imaginary parts
    x = rho_0[0]  # Assuming x is consistent for all files. This gives freq data
    real_data = rho_0[1] + rho_1[1] + rho_2[1]
    imag_data = rho_0[2] + rho_1[2] + rho_2[2]

    # Ensure all arrays have the same length (e.g., 3200)
    assert len(x) == len(real_data) == len(imag_data) == 3200, "Array dimensions mismatch!"

    all_results.append([x, rho_0[1], rho_1[1], rho_2[1], rho_0[2], rho_1[2], rho_2[2], real_data, imag_data])
    #all_results.append([x, real_data, imag_data]) Jiwoo's notation, Shape will be (14, 3, 3200)

# Convert all_results into a NumPy array
results_array = np.array(all_results)  

In [6]:
#now lets group as (14,3,3000)

output = []
for n in range(0, 14):  #
    
    average_result1 = (results_array[0+4*n][1] + results_array[1+4*n][1] + results_array[2+4*n][1] + results_array[3+4*n][1]) / 4
    average_result2 = (results_array[0+4*n][2] + results_array[1+4*n][2] + results_array[2+4*n][2] + results_array[3+4*n][2]) / 4
    average_result3 = (results_array[0+4*n][3] + results_array[1+4*n][3] + results_array[2+4*n][3] + results_array[3+4*n][3]) / 4
    
    output.append([average_result1, average_result2, average_result3])
    #all_results.append([x, real_data, imag_data]) Jiwoo's notation, Shape will be (14, 3, 3200)

# Convert all_results into a NumPy array
ns4_array= np.array(output)  
    

In [9]:
ns4_array.shape
    

(14, 3, 3200)